In [4]:
!pip install spacy pandas openpyxl flask


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
!spacy download it_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.9/567.9 MB 230.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_lg')


In [6]:
import spacy
import pandas as pd
import re
from spacy_entityruler import *

In [7]:
nlp = spacy.load('it_core_news_lg')

In [8]:
df = pd.read_excel('data//teams_data.xlsx')

In [14]:
team_names = df.name

In [9]:
add_labeled_entity_ruler_to_spacy_model(nlp, adjust_team_names_for_entity_ruler(team_names), 'TEAM')

NameError: name 'team_names' is not defined

In [10]:
with open('data/spacy_text.txt', 'r', encoding="utf8") as file:
    sample_text = file.read()
sample_text = re.sub(r'\n', ' ', sample_text)
sample_text = re.sub(r'\t', ' ', sample_text)

In [17]:
entity_ruler = nlp.get_pipe('entity_ruler')

In [18]:
pattern_money = [  
    {'label': 'MONEY', 'pattern': [{'IS_DIGIT':1},{'lemma':'milione'}]}, 
    {'label': 'MONEY', 'pattern': [{'IS_DIGIT':1},{'lemma':'milione'},{'POS':'ADP'},{'lower':'euro'}]},
    {'label': 'MONEY', 'pattern': [{'IS_DIGIT':1},{'lemma':'milione'},{'POS':'ADP'},{'lower':'dollari'}]},
    {'label': 'MONEY', 'pattern': [{'IS_DIGIT':1},{'lemma':'milione'},{'ORTH':'€'}]},
    {'label': 'MONEY', 'pattern': [{'IS_DIGIT':1},{'lemma':'milione'},{'ORTH':'$'}]},
    {'label': 'MONEY', 'pattern': [{'IS_DIGIT':1},{'lemma':'milione'},{'POS':'ADP'},{'lower':'euro'}]},
    {'label': 'MONEY', 'pattern': [{'IS_DIGIT':1},{'lemma':'milione'},{'POS':'ADP'},{'lower':'dollari'}]},
    {'label': 'MONEY', 'pattern': [{'IS_DIGIT':1},{'lemma':'mila'},{'ORTH':'€'}]},
    {'label': 'MONEY', 'pattern': [{'IS_DIGIT':1},{'lemma':'mila'},{'ORTH':'$'}]},
    {'label': 'MONEY', 'pattern': [{'IS_DIGIT':1},{'lower':'mln'}]},
    {'label': 'MONEY', 'pattern': [{'IS_DIGIT':1},{'lemma':'mln'},{'ORTH':'€'}]},
    {'label': 'MONEY', 'pattern': [{'IS_DIGIT':1},{'lemma':'mln'},{'ORTH':'$'}]}
]

pattern_sponsor = [
    {'label': 'SPONSOR', 'pattern': [{'LOWER': 'sponsor'},{'POS': 'NOUN'}]},
    {'label': 'SPONSOR', 'pattern': [{'POS': 'NOUN'},{'LOWER': 'sponsor'}]},
    {'label': 'SPONSOR', 'pattern': [{'LOWER': 'sponsor'},{'POS': 'ADJ'}]},
    {'label': 'SPONSOR', 'pattern': [{'LOWER': 'sponsorizzazione'}]}
]

entity_ruler.add_patterns(pattern_money)
entity_ruler.add_patterns(pattern_sponsor)

In [19]:
doc = nlp(sample_text)

In [20]:
money_sents = []
for sent in doc.sents:
    if any(ent.label_ == 'SPONSOR' for ent in sent.ents) & any(ent.label_ == 'MONEY' for ent in sent.ents):
        print(sent)
        for ent in sent.ents:
            print(ent.text, ent.label_)

Andando a fondo con le cifre incassate dal Milan tramite gli sponsor di maglia, possiamo partire dai circa 30 milioni di euro messi in cassa dal club rossonero tramite Puma, lo sponsor tecnico della squadra.
Milan TEAM
30 milioni di euro MONEY
Puma MISC
sponsor tecnico SPONSOR
mentre quello sulla spalla, BitMEX, porta nelle casse rossonere circa 5 milioni di euro, per un totale di 72 milioni di euro (circa) annuali.  Puma (Sponsor Tecnico) - 30 milioni di euro Emirates (Main Sponsor) - 30 milioni di euro WeFox (Sponsor retr maglia) - 7 milioni di euro BitMEX (Sponsor spalla) - 5 milioni di euro  Supera i 30 milioni il valore della maglia del Milan dal punto di vista commerciale.
BitMEX LOC
5 milioni di euro MONEY
72 milioni di euro MONEY
Puma MISC
Sponsor Tecnico SPONSOR
30 milioni di euro MONEY
Emirates ORG
Main Sponsor MISC
30 milioni di euro MONEY
WeFox MISC
Sponsor retr SPONSOR
7 milioni di euro MONEY
BitMEX MISC
Sponsor spalla SPONSOR
5 milioni di euro MONEY
Supera LOC
30 milioni 

In [21]:
#nlp.to_disk('model')